In [1]:
import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 
import itertools


## Working code

In [2]:
#REMOVING THE TIME FUNCTION COMPLETELY FROM THE BENDER'S DECOMPOSITION:


#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 13:00'
end_date='2018-05-28 13:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Since we are not looking at periods we give a single value to Solar_p: can create scenarios when algorithm works
Solar_p = 15
Market = 60

#--Constants--
Constants= {
    'Load_penalty':200, 
    'Hydro_cap':50, 
    'Load':150,
    'Market':60,
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return model.probs*(sum(model.yi[i] * model.p[i] for i in model.plants) - sum(model.Mi[i]*model.p[i] for i in model.plants) + model.alpha) #minimize(power produced - power sold to market)
    
    #return sum(model.yi[i] * model.p[i] for i in model.plants) + sum(model.ki[s] * model.phi[s] for s in model.solar) + sum(model.Li[n] * model.L_p[n] for n in model.penalty) + model.alpha

def p_bounds(model,i):
    return (model.Pmin[i], model.Pmax[i])   


def Hydro_capacity(model, i):
    return model.p[i] <= model.H_stage1[i]

def CreateCuts(model,c):
    return(model.alpha >= model.Phi[c] - sum(model.Lambda[c,i]*(model.p[i]-model.x_hat[c,i]) for i in model.plants))  #only takes into account the hydro power production scheduling from H_cap

#model.Phi[c] IS the Obj_2nd function result!. Need to include all the complicating variables with their sensitivities as well                                                                                                                      

# ----- Second stage ojbective and constraints --------------#
def Obj_2nd(model):
    return model.probs*(sum((model.yi[i]*model.p[i] + 0*model.phi + model.Li * model.L_p) for i in model.plants))
#p is a parameter in the second objective

def Solar_rule(model):
    return model.phi == Solar_p

def Hydro_capacity2(model, i):      #ensures that model.p does not use more capacity than what it has available
    return model.p[i] <= model.H_stage2[i]

def load_rule(model):
    return model.p['Hydro1'] + model.p['Hydro2'] + model.phi + model.L_p == model.L 

def calculate_initial_H_stage2(Constants, X_hat):
    initial_H_stage2 = {}
    for plant in X_hat:
        remaining_capacity = Constants['Hydro_cap'] - (X_hat[plant])
        initial_H_stage2[plant] = remaining_capacity
    return initial_H_stage2

# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts, Scenario):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants   =  Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')

    #Parameters
    model.Ci       =  pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.yi       =  pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin     =  pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax     =  pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage1 =  pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Initial capacity
    model.Mi       =  pyo.Param(model.plants, initialize=Constants['Market'])                           #market sell price
    model.L        =  pyo.Param(initialize=Constants['Load'])
    model.probs    =  pyo.Param(initialize=Scenario)

    # Variables
    model.p        =  pyo.Var(model.plants, bounds=p_bounds)  # Production from hydro plants

    # Constraints
    model.hydro_cons   =  pyo.Constraint(model.plants, rule=Hydro_capacity)

    # Cuts information
    model.Cut      =  pyo.Set(initialize = Cuts["Set"])
    model.Phi      =  pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda   =  pyo.Param(model.Cut, model.plants, initialize = Cuts["lambda"])
    model.x_hat    =  pyo.Param(model.Cut, model.plants, initialize = Cuts["x_hat"])
    
    model.alpha    =  pyo.Var(bounds = (-100000000,100000000))
    
    model.CreateCuts   =  pyo.Constraint(model.Cut, rule = CreateCuts)

    # Define objective function
    model.obj      =  pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model



def ModelSetUp_2nd(data, Constants, X_hat, rem_cap, Scenario):
    # Instance
    model          =  pyo.ConcreteModel()

    # Define sets
    model.plants   =  Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')

    # Define parameters
    model.X_hat    =  pyo.Param(model.plants, initialize=X_hat)
    model.Li       =  pyo.Param(initialize=Constants['Load_penalty']) 
    model.yi       =  pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin     =  pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax     =  pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage2 =  pyo.Param(model.plants, initialize=rem_cap)
    model.L        =  pyo.Param(initialize=Constants['Load'])
    model.probs    =  pyo.Param(initialize=Scenario)


    # Define variables
    model.p        =  pyo.Var(model.plants, bounds=p_bounds)
    model.phi      =  pyo.Var(within=NonNegativeReals)
    model.L_p      =  pyo.Var(within=NonNegativeReals)

    # Constraints
    model.load_cons    =  pyo.Constraint(rule=load_rule)
    model.solar_cons   =  pyo.Constraint(rule=Solar_rule)
    model.hydro_cons   =  pyo.Constraint(model.plants, rule=Hydro_capacity2)                                           #constraining power production from hydrom from the remaining capacity

    # Define objective function
    model.obj     =  pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        Cuts["lambda"][cut, i] = model.dual.get(model.hydro_cons[i])  # the lambda should be the economic change when we increase the 1st decision by 1. Should be something else
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}


scenarios =  {'High':1/3,'Avg':1/3,'Low':1/3}
solar_val_scenario = {'High':30,'Avg':15,'Low':0}

for s in scenarios:
    print()
    print('---------------------------------------------------------------------')
    print("Solving for scenario:",s)
    print('---------------------------------------------------------------------')
    Solar_p = solar_val_scenario[s]          #input of solar data to be used in the optimization
    print("The amount of solar power for scenario",s,"is:",Solar_p,"MW")
    #for i in range(3):
    tol = 1
    i = 0
    while abs(tol) > 0.1:
        i += 1
        print()
        print('Solving for iteration:',i)
        print()
        #Solve 1st stage problem
        m_1st = ModelSetUp_1st(data, Constants,Cuts,Scenario=scenarios[s])
        Solve(m_1st)
    
    
        # Process 1st stage result
        X_hat = {
                "Hydro1": m_1st.p["Hydro1"].value,
                "Hydro2": m_1st.p["Hydro2"].value,
            }
        
        display("How much capacity was used by each plant in stage 1 after solving loop: "  +  str( i ) + "  " + str(X_hat))
        #DisplayResults(m_1st)
        
        #Updating remaining capacity between stages
        initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat,)         #calculate changing capacity between stages
        display("How much capacity is left for each plant in stage 2 after solving loop: " +  str( i ) + "  " + str(initial_H_stage2))

        # Setup and solve 2nd stage problem with updated H_stage1
        m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2, Scenario=scenarios[s])      #must take into account the capacity used in stage 1 as a linking variable
        Solve(m_2nd)
        #DisplayResults(m_2nd)
        #Create new cuts for 1st stage problem
        Cuts = Cut_manage(Cuts,m_2nd)
        print()
        print()

        #Print results 2nd stage
        print("Objective function:",pyo.value(m_2nd.obj))
        print("Cut information acquired:")
        #for component in Cuts:
        #    if component == "lambda" or component == "x_hat":
        #        for j in m_2nd.plants:
        #            print(component,j,Cuts[component][i,j])
        #    else:
        #        print(component,Cuts[component])
        

        tol = (pyo.value(m_1st.alpha.value)-pyo.value(m_2nd.obj))
        print("TOLERANCE",abs(tol))
        #We perform a convergence check
        print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
        #input()
    
    print("")
    print('--------------------------------->','For scenario:',s,'the cost is:',m_2nd.obj(),'<---------------------------------')
    print("")


---------------------------------------------------------------------
Solving for scenario: High
---------------------------------------------------------------------
The amount of solar power for scenario High is: 30 MW

Solving for iteration: 1



"How much capacity was used by each plant in stage 1 after solving loop: 1  {'Hydro1': 40.0, 'Hydro2': 50.0}"

"How much capacity is left for each plant in stage 2 after solving loop: 1  {'Hydro1': 10.0, 'Hydro2': 0.0}"



Objective function: 14750.0
Cut information acquired:
TOLERANCE 100014750.0
UB: -100000000.0 - LB: 14750.0

Solving for iteration: 2



"How much capacity was used by each plant in stage 1 after solving loop: 2  {'Hydro1': 0.0, 'Hydro2': 0.0}"

...

"How much capacity is left for each plant in stage 2 after solving loop: 2  {'Hydro1': 23.999999999999996, 'Hydro2': 50.0}"



Objective function: 10916.666666666666
Cut information acquired:
TOLERANCE 1999.9999999950323
UB: 8916.666666671634 - LB: 10916.666666666666

Solving for iteration: 3



"How much capacity was used by each plant in stage 1 after solving loop: 3  {'Hydro1': 10.000000000000002, 'Hydro2': 0.0}"

"How much capacity is left for each plant in stage 2 after solving loop: 3  {'Hydro1': 40.0, 'Hydro2': 50.0}"



Objective function: 8916.666666666666
Cut information acquired:
TOLERANCE 4.967660061083734e-09
UB: 8916.666666671634 - LB: 8916.666666666666

---------------------------------> For scenario: Low the cost is: 8916.666666666666 <---------------------------------



In [4]:
# ----------------------- SDP ------------------------ #
scenarios =  {'High':1/3,'Avg':1/3,'Low':1/3}
solar_val_scenario = {'High':30,'Avg':15,'Low':0}

print(solar_val_scenario)

def SDP(Constants, scenarios, solar_val_scenario):
    Min = 0
    Max = Constants["Hydro_cap"]

    states_jump = 5
    List_states = [i for i in range(Min,Max+states_jump,states_jump)]

    Hydro1_initial_value = List_states
    Hydro2_initial_value = List_states
    
    List_combinations = [p for p in itertools.product(Hydro1_initial_value,Hydro2_initial_value)]

    
    """Start the SDP process"""
    
    #Pre-step: Formulate cut input data
    Cuts = {}
    Cuts["Set"] = []
    Cuts["Phi"] = {}
    Cuts["lambda"] = {}
    Cuts["x_hat"] = {}
    #For each combination we acquired
    for s in scenarios:
        print()
        print('---------------------------------------------------------------------')
        print("Solving for scenario:",s)
        print('---------------------------------------------------------------------')
        Solar_p = solar_val_scenario[s]          #input of solar data to be used in the optimization
        print("The amount of solar power for scenario",s,"is:",Solar_p,"MW")
        for initial_value in List_combinations:
            #Set 1st stage result
            X_hat = {
                "Hydro1": initial_value[0],
                "Hydro2": initial_value[1],
                }
            
            initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)    
            if sum(X_hat[i] for i in X_hat) <= Max:
                m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2, Scenario=scenarios[s])     
                Solve(m_2nd)
                #DisplayResults(m_2nd)
                Cuts = Cut_manage(Cuts,m_2nd)
            else:
                pass
        
        #Solve the 1st stage problem with the acquired cuts
        m_1st = ModelSetUp_1st(data, Constants,Cuts, Scenario=scenarios[s])
        Solve(m_1st)
        
        X_hat = {
                "Hydro1": m_1st.p["Hydro1"],
                "Hydro2": m_1st.p["Hydro2"],
            }

        #Print results 1st stage
        for x in X_hat:
            print(x,X_hat[x].value)
        print(pyo.value(m_1st.alpha.value))
        

    return()
SDP(Constants, scenarios, Solar_p)

{'High': 30, 'Avg': 15, 'Low': 0}

---------------------------------------------------------------------
Solving for scenario: High
---------------------------------------------------------------------


TypeError: 'int' object is not subscriptable

In [5]:
Min = 0
Max = Constants["Hydro_cap"]

states_jump = 5
List_states = [i for i in range(Min,Max+states_jump,states_jump)]

Hydro1_initial_value = List_states
Hydro2_initial_value = List_states
    
List_combinations = [p for p in itertools.product(Hydro1_initial_value,Hydro2_initial_value)]

    
"""Start the SDP process"""
    
#Pre-step: Formulate cut input data
Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}
#For each combination we acquired
for s in scenarios:
    print()
    print('---------------------------------------------------------------------')
    print("Solving for scenario:",s)
    print('---------------------------------------------------------------------')
    Solar_p = solar_val_scenario[s]          #input of solar data to be used in the optimization
    print("The amount of solar power for scenario",s,"is:",Solar_p,"MW")
    for initial_value in List_combinations:
        #Set 1st stage result
        X_hat = {
            "Hydro1": initial_value[0],
            "Hydro2": initial_value[1],
            }
            
        initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)    
        if sum(X_hat[i] for i in X_hat) <= Max:
            m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2, Scenario=scenarios[s])     
            Solve(m_2nd)
            #DisplayResults(m_2nd)
            Cuts = Cut_manage(Cuts,m_2nd)
        else:
            pass
        
        #Solve the 1st stage problem with the acquired cuts
    m_1st = ModelSetUp_1st(data, Constants,Cuts, Scenario=scenarios[s])
    Solve(m_1st)
        
    X_hat = {
            "Hydro1": m_1st.p["Hydro1"],
            "Hydro2": m_1st.p["Hydro2"],
        }

    #Print results 1st stage
    for x in X_hat:
        print(x,X_hat[x].value)
    print(pyo.value(m_1st.alpha.value))


---------------------------------------------------------------------
Solving for scenario: High
---------------------------------------------------------------------
The amount of solar power for scenario High is: 30 MW
Hydro1 9.999999999999986
Hydro2 0.0
4916.666666671634

---------------------------------------------------------------------
Solving for scenario: Avg
---------------------------------------------------------------------
The amount of solar power for scenario Avg is: 15 MW
Hydro1 9.999999999999986
Hydro2 0.0
6916.666666671634

---------------------------------------------------------------------
Solving for scenario: Low
---------------------------------------------------------------------
The amount of solar power for scenario Low is: 0 MW
Hydro1 9.999999999999986
Hydro2 0.0
8916.666666671634


In [ ]:
def SDP():
    Min = 0
    Max = Constants["Hydro_cap"]

    states_jump = 5
    List_states = [i for i in range(Min,Max+states_jump,states_jump)]

    Hydro1_initial_value = List_states
    Hydro2_initial_value = List_states
    
    List_combinations = [p for p in itertools.product(Hydro1_initial_value,Hydro2_initial_value)]

    
    """Start the SDP process"""
    
    #Pre-step: Formulate cut input data
    Cuts = {}
    Cuts["Set"] = []
    Cuts["Phi"] = {}
    Cuts["lambda"] = {}
    Cuts["x_hat"] = {}
    #For each combination we acquired
    for initial_value in List_combinations:
        #Set 1st stage result
        X_hat = {
            "Hydro1": initial_value[0],
            "Hydro2": initial_value[1],
            }
        
        #display("How much capacity was used by each plant in stage 1 after solving loop: "  +  str( i ) + "  " + str(X_hat))

        initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)    
           
        if sum(X_hat[i] for i in X_hat) <= Max:
            m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2)     
            Solve(m_2nd)
            #DisplayResults(m_2nd)
            Cuts = Cut_manage(Cuts,m_2nd)
    
        #If planted is higher than allowed
        else:
            pass
    
    #Solve the 1st stage problem with the acquired cuts
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
    
    X_hat = {
            "Hydro1": m_1st.p["Hydro1"],
            "Hydro2": m_1st.p["Hydro2"],
        }

    #Print results 1st stage
    for x in X_hat:
        print(x,X_hat[x].value)
    print(pyo.value(m_1st.alpha.value))
    

    return()
SDP()

TypeError: ModelSetUp_2nd() missing 1 required positional argument: 'Scenario'

## Testing code

#REMOVING THE TIME FUNCTION COMPLETELY FROM THE BENDER'S DECOMPOSITION:


#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 13:00'
end_date='2018-05-28 13:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Since we are not looking at periods we give a single value to Solar_p: can create scenarios when algorithm works
Solar_p = 10

#Unique scenarios for stochastic variable
S_high=Solar_p*1.5
S_avg=Solar_p*1
S_low=Solar_p*0.5

#Load 
#L= {1:50} #3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

Market = 30

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':50, 
    'Load':150,
    'probs':{'S_high':1/3, 'S_avg':1/3, 'S_low':1/3},    
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i] for i in model.plants) - sum(model.m[i]*model.p[i] for i in model.plants) #minimize(power produced - power sold to market)
    
def p_bounds(model,i):
    return (model.Pmin[i], model.Pmax[i])   


def Hydro_capacity(model, i):
    return model.p[i] <= model.H_stage1[i]

def CreateCuts(model,c):
    return(model.alpha >= model.Phi[c] + sum(model.Lambda[c,i]*(model.p[i]-model.x_hat[c,i]) for i in model.plants))


# ----- Second stage ojbective and constraints --------------# -- IMPLEMENT scenarios to have Stochasticity in the second stage decision.
def Obj_2nd(model):
    return sum(model.probs[l] * (model.yi[i]*model.p[i] + model.phi[l] + model.Li[n] * model.L_p[n]) for i in model.plants for l in model.scenarios for n in model.penalty)

#model.ki[s] *

#def Solar_rule(model,l):
#    return model.phi['Solar',l] == Solar_p

def Solar_high(model):
    if model.probs['S_high']>0:
        return model.phi['S_high']==S_high
    else:
        return model.phi['S_high']==0
    
#Solar production in stage 2 must be equal to scenario for average production
def Solar_avg(model):
    if model.probs['S_avg']>0:
        return  model.phi['S_avg'] == S_avg
    else:
        return  model.phi['S_avg'] == 0
    
#Solar production in stage 2 must be equal to scenario for low production
def Solar_low(model):
    if model.probs['S_low']>0:
        return  model.phi['S_low'] == S_low
    else:
        return  model.phi['S_low'] == 0
    

def Hydro_capacity2(model, i):      #ensures that model.p does not use more capacity than what it has available
    return model.p[i] <= model.H_stage2[i]

def load_rule(model,l):
    return model.p['Hydro1'] + model.p['Hydro2'] + model.phi['Solar',l] + model.L_p['Load_penalty'] == model.L 

def calculate_initial_H_stage2(Constants, X_hat):
    initial_H_stage2 = {}
    for plant in X_hat:
        remaining_capacity = Constants['Hydro_cap'] - (X_hat[plant])
        initial_H_stage2[plant] = remaining_capacity
    return initial_H_stage2

# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')

    #Parameters
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage1 = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Initial capacity
    model.m = pyo.Param(model.plants, initialize=Market)   #market sell price

    model.L = pyo.Param(initialize=Constants['Load'])


    # Variables
    model.p   = pyo.Var(model.plants, bounds=p_bounds)  # Production from hydro plants

    # Constraints
    model.hydro_cons=pyo.Constraint(model.plants, rule=Hydro_capacity)

    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.plants, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.plants, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)


    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model


def ModelSetUp_2nd(data, Constants, X_hat, rem_cap):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    #model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar') ->used for solar cost
    model.penalty= Set(initialize=['Load_penalty']) 
    model.scenarios = Set(initialize=Constants['probs'].keys())

    # Define parameters
    model.X_hat = pyo.Param(model.plants, initialize=X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    #model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])  #solar is free and will always be utilizied
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    #model.Phi_min=pyo.Param(model.scenarios, initialize=data['solar']['P_min'])
    #model.Phi_max=pyo.Param(model.scenarios, initialize=data['solar']['P_max'])
    model.H_stage2 = pyo.Param(model.plants, initialize=rem_cap)
    model.probs=pyo.Param(model.scenarios, initialize=Constants['probs'])       #Scenarios indexed by "l"
    model.L = pyo.Param(initialize=Constants['Load'])
    
    # Define variables
    model.p   = pyo.Var(model.plants, bounds=p_bounds)
    model.phi = pyo.Var(model.scenarios, within=NonNegativeReals)
    model.L_p = pyo.Var(model.penalty, within=NonNegativeReals)

    #Constraints
    model.high_sun= pyo.Constraint(rule=Solar_high)
    model.avg_sun= pyo.Constraint(rule=Solar_avg)
    model.low_sun= pyo.Constraint(rule=Solar_low)

    model.load_cons  = pyo.Constraint(model.scenarios, rule=load_rule)
    model.hydro_cons = pyo.Constraint(model.plants, rule=Hydro_capacity2)  #constraining power production from hydrom from the remaining capacity
    
    #Stochastic constraints

    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        Cuts["lambda"][cut, i] = model.dual.get(model.hydro_cons[i])  # the lambda should be the economic change when we increase the 1st decision by 1, => dual value of hydro prod
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}

#X_hat = {
#    "Hydro1": 0.0,
#    "Hydro2": 0.0,
#}

#This is the while-loop in principle, but for this case is only a for-loop

#CODE FOR LATER WHEN LOGIC WORKS:
#tol = 1e-4     #tolerance for while loop
#while (pyo.value(m_1st.alpha.value - pyo.value(m_2nd.obj)) >= tol): #while loop that doesn't end until upper bound and lower bound converge (not working so something similar)



#For now:
for i in range(5):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {
            "Hydro1": m_1st.p["Hydro1"].value,
            "Hydro2": m_1st.p["Hydro2"].value,

        }
    display(X_hat)
    DisplayResults(m_1st)
    
    #Updating remaining capacity between stages
    initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)         #calculate changing voluem
    display("How much capacity is left for each plant for stage 2 after solving loop: " +  str( i ) + "  " + str(initial_H_stage2))

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)
    DisplayResults(m_2nd)
    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    print()
    print()

    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    #input()
